# Система. Датасет.

На прошлом занятии мы узнали, каким образом мы можем загрузить картинку, представив ее в виде тензора. Сегодня мы научимся считывать большое количество картинок, делать это оптимально (в несколько потоков), используя имя файлов или папок как таргет (от англ. "target" - "цель", то есть класс картинки), и применять аугментацию "на лету". 

<img style="width:40%" src="./images/os.jpg">

Для начала познакомимся с библиотекой os. Из названия можно догататься, что эта библиотека позволяет взаимодействовать с операционной системой: создавать и удалять файлы и папки, менять права пользователей на взаимодействие с ними, вне зависимости от операционной системы, создавать пути к файлам, узнавать пути к рабочим директориям и много другое.

## OS

In [1]:
import os

Было сказано, что os способна создавать пути к файлам вне зависимости от операционной системы. Что это значит? В unix-подобных системах: MacOS, Linux, Ubuntu - путь к файлам разделяется прямым слэшем: /home/user/data/image.png. Но, например, в windows разделителем является обратный слэш и путь к файлу может выглядеть иначе: C\User\data\image.png. Давайте попробуем создать путь до картинки с енотом из прошлой лекции. В зависимости от ОС (Операционной Системы), результат будет разным. Мы знаем что картинка лежит в папке **images**, а название картинки **racoon.jpg**. 

Для этого нам понадобится модуль path и функция join( ) из него. 

join(a, *p) - принимает через запятую названия папок и имя файла, компонуя из них путь к файлу.

In [2]:
path_to_racoon = os.path.join('images', 'raccoon.jpg')

path_to_racoon

'images/raccoon.jpg'

Отлично, а теперь представим что у нас есть файл, который находится где-то глубоко в папках, создадим путь до него (путь выдуманный, можете скомпоновать другой на свое усмотрение).

In [3]:
random_path = os.path.join('bucket', 'ml', 'data', 'file.txt')

random_path

'bucket/ml/data/file.txt'

Теперь научимся доставать каждую часть пути отдельно. Чтобы из всего пути выделить только название файла, можно воспользоваться функцией os.path.basename( ).

In [4]:
os.path.basename(random_path)

'file.txt'

Чтобы быстро извлечь название папки на любом уровне, удобнее будеть использовать метод строки .split( ). Разделять будем либо по '/', либо по '\', в зависимости от системы. Чтобы наша реализация была кроссплатформенная (независима от ОС) можно выявить какой разделитель используется при помощи свойства os.path.sep (сокращение от "separator" - "разделитель")

In [5]:
os.path.sep

'/'

In [6]:
random_path.split(os.path.sep)

['bucket', 'ml', 'data', 'file.txt']

Например, название конечной папки - это элемент данного масссива по индексу [-2].

In [7]:
random_path.split(os.path.sep)[-2]

'data'

## glob, os.listdir

Довольно часто мы либо не помним как называются файлы, лежащие в определенной директории, либо хотим проитерироваться по всем находящимся в папке файлам. В обоих случаях мы хотим получить список файлов, находящихся в определенной директории. Сделать это позволяет библиотека glob или функция os.listdir( )

Давайте посмотрим что лежит в папке data/cat_vs_dogs

In [ ]:
from glob import glob

In [ ]:
path_to_files = os.path.join('data', 'cats_vs_dogs')

In [ ]:
os.listdir(path_to_files)

Точно такой же результат можно получить при помощи функции glob из модуля glob (да, названия одинаковые :). Только эта функция не просто перечисляет файлы в директории, а способна находить те из них, которые соответствуют определенному паттерну. Например, если мы хотим вывести только файлы со словом 'dog' в названии, то мы можем использовать паттерн 'dog*'. Звездочка в пути означает "тут может находиться что угодно". Проще понять на примере.

In [ ]:
pattern = os.path.join('data', 'cats_vs_dogs', 'dog*')

pattern

In [ ]:
glob(pattern)

Как можно наблюдать, glob выводит не только список файлов в директории, но и полные пути к ним.

#### Задание

С помощью полученных навыков из этого и предыдущих занятий, создайте два массива: images и targets - массив с тензорами и массив с таргетами. Создайте их таким образом, чтобы каждый тензор являлся отдельной картинкой и чтобы между массивами было соответствие: images[idx] содержал картинку с таргетом target[idx], где idx - индекс.

In [ ]:
images = []
targets = []

# Ваш код здесь
#
#

## Dataset

Теперь у нас есть два списка: один с картинками, а второй с таргетами (классами). Теперь, если мы хотим получить пару: картинка, класс - то нам надо сначала определить индекс картинки, которая нас интересует, а потом извлечь картинку из массива images отдельно, извлечь таргет из массива targets отдельно. Сделать этот процесс удобнее позволит встроенная функция zip, которая "сшивает" списки, почти также как собачка на куртке соединяет зубья молнии. Но если мы захотим применять аугментацию, перемешивать датасет, подготавливать картинки параллельно, то столкнемся с потребностью изучать сторонние библиотеки и писать много кода. Идеальным решением будет использовать класс DataLoader из torch.utils.data

In [ ]:
import cv2

class MyDataset():
    def __init__(self, images, targets):
        self.images = images
        self.targets = targets
        
    
    def __getitem__(self, idx):
        img = self.images[idx]
        img = cv2.resize(img, (256, 256))
        
        tgt = self.targets[idx]
        return img, tgt
    
    
    def __len__(self):
        return len(self.images)

In [ ]:
dataset = MyDataset(images, targets)

In [ ]:
for img, tgt in dataset:
    print(img.shape, tgt)

### Dataset + augmentation

In [ ]:
import albumentations as alb

In [ ]:
class MyDataset():
    def __init__(self, images, targets, transforms):
        self.images = images
        self.targets = targets
        self.transforms = transforms
        
    
    def __getitem__(self, idx):
        img = self.images[idx]
        img = cv2.resize(img, (256, 256))
        img = self.transforms(image=img)['image']
        
        tgt = self.targets[idx]
        return img, tgt
    
    
    def __len__(self):
        return len(self.images)

In [ ]:
transforms = alb.Compose([
    alb.RandomBrightness(limit=0.5, always_apply=False, p=0.5), 
    alb.Rotate(limit=30, p=0.5)
])

In [ ]:
dataset = MyDataset(images, targets, transforms)

### Dataset + albumentations + чтение картинки

Теперь инициализатор (конструктор) принимает на вход лишь пути к файлам с картинками, а сама картинки будет подгружаться во время итерирования по датасету.

In [ ]:
class MyDataset():
    def __init__(self, paths, transforms):
        self.paths = paths
        self.transforms = transforms
        
    
    def __getitem__(self, idx):
        img = None
        tgt = None
        
        # тут вы должны считать картинку и определить
        # ее таргет
        
        return img, tgt
    
    
    def __len__(self):
        return len(self.paths)

## DataLoader

DataLoader позволяет распараллелить процесс подгрузки данных, подгружая картинки сразу батчами, при этом перемешивая датасет каждый раз (если установить флаг shuffle=True).

In [ ]:
from torch.utils.data import DataLoader

In [ ]:
loader = DataLoader(
    dataset=dataset, # объект с методами __getitem__() , __len__()
    batch_size=2,   # размер батча
    shuffle=True,  # если True, то датасет будет перемешиваться
    num_workers=4 # кол-во потоков, которое можно выделить для параллельной подгрузки данных
                   )

In [ ]:
for batch_imgs, batch_targets in loader:
    print(batch_imgs.shape, batch_targets)